In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 13) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(epochs = 300000, npoints = 24, batchsize = 1000, nsamples = 8) #greedyprob - 3, with +ve reinforcement

147.6300048828125 -39.130165100097656 -42.130638122558594 -38.727020263671875 -41.327423095703125 -41.327423095703125
145.8141632080078 -37.553794860839844 -41.73293685913086 -38.403411865234375 -42.569644927978516 -42.56964874267578
145.2933349609375 -36.05746841430664 -41.14223098754883 -35.894775390625 -40.12419891357422 -40.12419891357422
144.07333374023438 -33.18872833251953 -39.58185958862305 -34.03119659423828 -38.28373718261719 -38.28373336791992
141.55833435058594 -34.21745300292969 -40.15219497680664 -35.32804489135742 -43.45423126220703 -43.45423126220703
140.19000244140625 -35.605140686035156 -40.90610122680664 -35.6597900390625 -39.729713439941406 -39.729713439941406
140.43833923339844 -35.70671844482422 -40.92210388183594 -35.89508056640625 -41.93522644042969 -41.93522644042969
139.77166748046875 -34.98406982421875 -40.59482192993164 -35.44009780883789 -42.14259719848633 -42.142601013183594
139.24000549316406 -33.625247955322266 -39.74009704589844 -32.95680618286133 -36.5

137.74501037597656 -32.73765182495117 -39.478485107421875 -32.06786346435547 -41.17770767211914 -41.17770004272461
137.90000915527344 -33.68585205078125 -39.946510314941406 -33.3446044921875 -41.48806381225586 -41.48806381225586
137.9116668701172 -34.19501876831055 -40.13987350463867 -35.12320327758789 -38.898826599121094 -38.89883041381836
138.1800079345703 -34.28532409667969 -40.29047393798828 -33.15214538574219 -39.74622344970703 -39.746219635009766
137.16416931152344 -33.982547760009766 -40.22073745727539 -33.40567398071289 -39.25687789916992 -39.25688171386719
137.72999572753906 -33.55441665649414 -40.04896926879883 -33.63929748535156 -41.329227447509766 -41.329227447509766
136.99000549316406 -33.30035400390625 -39.889122009277344 -30.398706436157227 -42.127742767333984 -42.127742767333984
137.55751037597656 -33.43194580078125 -39.84518814086914 -31.626811981201172 -37.862892150878906 -37.86288833618164
138.31666564941406 -33.69757080078125 -39.90480422973633 -32.61186981201172 -4

136.88833618164062 -33.522552490234375 -39.787166595458984 -33.93061065673828 -37.37055969238281 -37.37055969238281
137.13417053222656 -33.449703216552734 -39.85860061645508 -31.83376121520996 -41.01161193847656 -41.01161193847656
137.27749633789062 -33.2744140625 -39.75217819213867 -33.12343215942383 -41.23526382446289 -41.235267639160156
136.75750732421875 -33.26393127441406 -39.77207565307617 -32.32054901123047 -36.11711883544922 -36.11711502075195
136.44667053222656 -33.327964782714844 -39.806068420410156 -32.72517395019531 -41.053157806396484 -41.053157806396484
136.9408416748047 -33.469078063964844 -39.93690490722656 -33.30502700805664 -45.5419921875 -45.5419921875
137.30166625976562 -33.42633056640625 -39.92208480834961 -33.89165496826172 -39.89262771606445 -39.89263153076172
137.850830078125 -33.34645080566406 -39.85653305053711 -32.37153625488281 -39.508384704589844 -39.50838851928711
137.47666931152344 -33.19594955444336 -39.727046966552734 -32.89775085449219 -40.029014587402

135.6316680908203 -32.32547378540039 -39.26726150512695 -31.555179595947266 -38.58395004272461 -38.58395004272461
135.87083435058594 -32.618202209472656 -39.4354362487793 -31.89982795715332 -41.28460693359375 -41.28460693359375
135.9875030517578 -33.242088317871094 -39.884490966796875 -32.442352294921875 -39.4537353515625 -39.45372772216797
136.9216766357422 -34.06096267700195 -40.34113693237305 -35.18412780761719 -37.98333740234375 -37.983341217041016
137.3050079345703 -34.442752838134766 -40.549190521240234 -33.66801452636719 -41.151912689208984 -41.151912689208984
137.4425048828125 -34.440574645996094 -40.51409912109375 -32.78266906738281 -41.444061279296875 -41.444068908691406
137.600830078125 -34.13017272949219 -40.31702423095703 -34.12181854248047 -36.712303161621094 -36.712303161621094
138.0016632080078 -33.72576904296875 -40.050113677978516 -33.26502227783203 -38.246498107910156 -38.246498107910156
137.95750427246094 -33.29362487792969 -39.88918685913086 -32.50322723388672 -36.

136.52667236328125 -34.34878158569336 -40.740482330322266 -34.116477966308594 -39.11510467529297 -39.11510467529297
136.14584350585938 -34.37774658203125 -40.72273254394531 -34.15751266479492 -40.15766525268555 -40.15766525268555
136.16749572753906 -34.076393127441406 -40.571510314941406 -33.87743377685547 -39.45592498779297 -39.45592498779297
135.8574981689453 -33.455482482910156 -40.17865753173828 -32.18229675292969 -40.644840240478516 -40.64484405517578
136.12417602539062 -32.86936950683594 -39.77353286743164 -33.10124969482422 -38.87709426879883 -38.87709045410156
136.34083557128906 -32.533203125 -39.579063415527344 -32.21303939819336 -40.0108642578125 -40.0108642578125
136.61666870117188 -32.334381103515625 -39.41911315917969 -34.01506042480469 -37.48280334472656 -37.4827995300293
136.1316680908203 -32.631351470947266 -39.620513916015625 -33.229698181152344 -37.461055755615234 -37.461055755615234
136.53416442871094 -33.142250061035156 -39.916114807128906 -34.19577407836914 -37.708

...

130.38333129882812 -34.026458740234375 -40.71820831298828 -33.7563591003418 -40.25056076049805 -40.250553131103516
130.87001037597656 -34.012996673583984 -40.609336853027344 -34.29658126831055 -40.96335983276367 -40.96335983276367
130.80084228515625 -33.73915100097656 -40.44698715209961 -34.326690673828125 -38.42995071411133 -38.42995071411133
131.4116668701172 -33.45381546020508 -40.358394622802734 -32.3363037109375 -40.103492736816406 -40.103492736816406
131.33750915527344 -33.19568634033203 -40.20935821533203 -33.98715591430664 -43.096187591552734 -43.09618377685547
131.72750854492188 -33.1395149230957 -40.1798210144043 -33.67885971069336 -36.95627975463867 -36.95627975463867
130.96749877929688 -33.14952087402344 -40.23842239379883 -33.03437042236328 -42.1236686706543 -42.12367630004883
130.9250030517578 -33.338809967041016 -40.337188720703125 -33.540000915527344 -38.86493682861328 -38.86494064331055
130.3800048828125 -33.49967575073242 -40.433135986328125 -34.510780334472656 -36.25

131.0800018310547 -33.53227996826172 -40.490455627441406 -32.583805084228516 -44.251678466796875 -44.25168228149414
131.2375030517578 -33.369075775146484 -40.414146423339844 -32.74562454223633 -39.58814239501953 -39.588138580322266
130.8733367919922 -33.210182189941406 -40.334449768066406 -34.44007110595703 -36.05342102050781 -36.05342102050781
131.15333557128906 -33.091522216796875 -40.29844284057617 -34.6590690612793 -40.448150634765625 -40.448150634765625
131.02667236328125 -33.0803337097168 -40.24699020385742 -33.18286895751953 -38.38188171386719 -38.38188552856445
131.15167236328125 -33.01012420654297 -40.21971893310547 -32.895713806152344 -44.900508880615234 -44.900508880615234
130.9041748046875 -33.1725959777832 -40.200801849365234 -34.054832458496094 -35.053192138671875 -35.05319595336914
130.61416625976562 -33.558677673339844 -40.392784118652344 -33.053070068359375 -38.99104309082031 -38.99104309082031
130.086669921875 -33.96792984008789 -40.632083892822266 -34.34360122680664 

130.35333251953125 -33.44894790649414 -40.43110656738281 -32.23597717285156 -41.35697555541992 -41.35697555541992
130.5308380126953 -33.61791229248047 -40.560523986816406 -32.256080627441406 -40.519256591796875 -40.51926040649414
130.461669921875 -33.6790885925293 -40.560447692871094 -33.73353576660156 -39.887413024902344 -39.88741683959961
130.24166870117188 -33.80789566040039 -40.68330764770508 -32.701622009277344 -39.102577209472656 -39.10258102416992
130.163330078125 -33.65795135498047 -40.59278106689453 -34.09241485595703 -39.67945098876953 -39.67945098876953
130.475830078125 -33.554439544677734 -40.567134857177734 -34.02330017089844 -41.19269943237305 -41.19270324707031
130.51333618164062 -33.49482345581055 -40.52117919921875 -33.36326599121094 -40.90624237060547 -40.90624237060547
130.30833435058594 -33.349334716796875 -40.517051696777344 -32.127647399902344 -43.01565170288086 -43.01565170288086
130.7566680908203 -33.39012145996094 -40.502342224121094 -34.08429718017578 -41.6118

129.22250366210938 -33.328338623046875 -40.36691665649414 -32.29489517211914 -38.62757873535156 -38.62757873535156
129.625 -33.52581787109375 -40.44501876831055 -33.31185531616211 -39.30109786987305 -39.30109786987305
129.22500610351562 -33.62981033325195 -40.547752380371094 -35.180084228515625 -42.40922546386719 -42.40922546386719
129.9566650390625 -33.787071228027344 -40.62484359741211 -34.361412048339844 -40.51169967651367 -40.51169967651367
129.8941650390625 -33.74748611450195 -40.639732360839844 -32.03447341918945 -36.8848876953125 -36.8848876953125
129.99082946777344 -33.6892204284668 -40.63444519042969 -33.18265151977539 -40.816802978515625 -40.81680679321289
129.82916259765625 -33.62570571899414 -40.60183334350586 -32.862335205078125 -43.9847412109375 -43.984737396240234
129.57833862304688 -33.53240203857422 -40.48663330078125 -32.42223358154297 -43.71794128417969 -43.717933654785156
129.6366729736328 -33.60279846191406 -40.50445556640625 -32.307125091552734 -40.171627044677734

130.01666259765625 -33.3302001953125 -40.42185592651367 -33.57084655761719 -36.17441177368164 -36.17441177368164
130.40249633789062 -33.44781494140625 -40.571434020996094 -33.487152099609375 -40.500694274902344 -40.50069046020508
129.54916381835938 -33.508785247802734 -40.61870574951172 -32.8769416809082 -42.39347839355469 -42.39348220825195
130.0291748046875 -33.74678421020508 -40.72661590576172 -33.77825927734375 -37.22148513793945 -37.22148895263672
129.7933349609375 -33.78615188598633 -40.75453186035156 -35.1534309387207 -42.163970947265625 -42.16397476196289
129.5500030517578 -33.67128372192383 -40.6192626953125 -33.90234375 -42.23081970214844 -42.23081588745117
129.88417053222656 -33.51314926147461 -40.59133529663086 -33.832767486572266 -39.757286071777344 -39.757286071777344
129.4108428955078 -33.3330192565918 -40.461673736572266 -33.591651916503906 -43.44657897949219 -43.44657516479492
129.84750366210938 -33.325626373291016 -40.500083923339844 -34.22529983520508 -39.55004882812

In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= 0
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 25) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(npoints = 54, batchsize = 500, nsamples = 8)

537.6380004882812 -138.98419189453125 -147.93531799316406 -146.45074462890625 -149.61941528320312 -149.61941528320312
532.5540161132812 -138.10366821289062 -147.8144989013672 -146.654541015625 -150.35662841796875 -150.3566436767578
530.2640380859375 -136.0601348876953 -147.40367126464844 -146.42555236816406 -147.881103515625 -147.88108825683594
525.9000244140625 -134.02139282226562 -146.74395751953125 -145.3341064453125 -148.57504272460938 -148.5750274658203
524.9900512695312 -132.3279571533203 -146.12123107910156 -145.25173950195312 -148.4559783935547 -148.45594787597656
524.1719970703125 -131.33157348632812 -145.742431640625 -141.67303466796875 -142.32098388671875 -142.32098388671875
521.322021484375 -130.02227783203125 -145.23062133789062 -144.06594848632812 -145.3900146484375 -145.38998413085938
519.8140258789062 -127.49968719482422 -144.056640625 -144.09527587890625 -144.72189331054688 -144.72189331054688
521.4620361328125 -124.63572692871094 -142.46963500976562 -139.2878875732422

514.114013671875 -122.0224380493164 -140.5898895263672 -144.7041778564453 -137.571533203125 -137.57154846191406
514.2300415039062 -122.39403533935547 -140.8399200439453 -143.66539001464844 -143.32257080078125 -143.32257080078125
513.634033203125 -123.73143005371094 -141.79444885253906 -141.58401489257812 -133.7679443359375 -133.76792907714844
517.447998046875 -125.28252410888672 -142.63766479492188 -143.10848999023438 -138.99832153320312 -138.99832153320312
513.8720092773438 -125.713134765625 -142.92547607421875 -147.26687622070312 -140.062744140625 -140.062744140625
515.4739990234375 -125.82928466796875 -142.84996032714844 -147.9103240966797 -138.15269470214844 -138.15269470214844
514.2980346679688 -125.21936798095703 -142.58120727539062 -143.18414306640625 -144.5225372314453 -144.5225372314453
514.5540161132812 -124.11746215820312 -141.89398193359375 -136.3398895263672 -138.9828338623047 -138.9828338623047
509.9980163574219 -122.69064331054688 -140.72781372070312 -141.65386962890625 

517.2360229492188 -121.35770416259766 -139.65931701660156 -140.25885009765625 -140.28668212890625 -140.2866668701172
515.9640502929688 -120.6246337890625 -139.36024475097656 -138.7896270751953 -140.9609375 -140.9609375
516.5420532226562 -121.35859680175781 -139.78598022460938 -133.77322387695312 -144.1346893310547 -144.13470458984375
514.6600341796875 -123.0822525024414 -141.15008544921875 -142.89846801757812 -141.82235717773438 -141.8223419189453
516.8980102539062 -124.23226928710938 -141.9872589111328 -146.69873046875 -144.220458984375 -144.22042846679688
519.35205078125 -124.6902847290039 -142.33163452148438 -143.93702697753906 -140.84738159179688 -140.84738159179688
516.6680297851562 -123.83184814453125 -141.8417510986328 -144.85797119140625 -148.1279296875 -148.12789916992188
515.9860229492188 -122.53752136230469 -141.01405334472656 -151.07037353515625 -131.84259033203125 -131.84255981445312
515.2900390625 -121.79325866699219 -140.52713012695312 -136.75643920898438 -140.2553710937

517.6420288085938 -118.97877502441406 -138.24066162109375 -130.60134887695312 -134.1087646484375 -134.10874938964844
518.738037109375 -124.51573181152344 -141.74757385253906 -142.69053649902344 -136.154541015625 -136.15457153320312
515.8240356445312 -128.2661590576172 -144.0184326171875 -140.3527374267578 -146.35922241210938 -146.3592529296875
517.842041015625 -130.12281799316406 -144.99697875976562 -148.08236694335938 -148.73707580566406 -148.73707580566406
521.8380126953125 -130.74729919433594 -145.2322235107422 -146.12693786621094 -148.87521362304688 -148.87521362304688
519.666015625 -130.55978393554688 -145.09498596191406 -144.96151733398438 -146.13644409179688 -146.13645935058594
517.1280517578125 -129.133544921875 -144.44830322265625 -140.9606475830078 -145.4196319580078 -145.41964721679688
517.5040283203125 -126.37647247314453 -142.95347595214844 -141.06201171875 -146.38568115234375 -146.3856658935547
514.2920532226562 -122.53421020507812 -140.69271850585938 -146.35997009277344 

517.4120483398438 -123.82118225097656 -141.38987731933594 -146.89895629882812 -141.45980834960938 -141.4597625732422
518.822021484375 -123.80950927734375 -141.34495544433594 -137.90994262695312 -148.436279296875 -148.43624877929688
516.248046875 -123.26822662353516 -140.9853515625 -142.35763549804688 -142.51638793945312 -142.51641845703125
518.2140502929688 -122.01557922363281 -140.2266082763672 -139.44943237304688 -134.66729736328125 -134.66726684570312
518.3680419921875 -120.83914184570312 -139.39801025390625 -135.07568359375 -138.61094665527344 -138.6109161376953
517.5 -119.63032531738281 -138.55772399902344 -139.40579223632812 -141.53213500976562 -141.5321044921875
517.9000244140625 -118.58738708496094 -137.7484130859375 -137.62754821777344 -143.09652709960938 -143.09654235839844
516.5680541992188 -118.34347534179688 -137.69622802734375 -141.22186279296875 -129.09507751464844 -129.09506225585938
515.93603515625 -119.19261169433594 -138.05267333984375 -138.06590270996094 -133.354431

...

474.74603271484375 -118.51860809326172 -135.88931274414062 -139.9208984375 -139.90948486328125 -139.9095001220703
475.1040344238281 -118.90074920654297 -135.956298828125 -124.94137573242188 -119.06703186035156 -119.0670394897461
476.988037109375 -119.56273651123047 -136.3390350341797 -115.97672271728516 -144.2639617919922 -144.2639923095703
474.8620300292969 -117.9942398071289 -136.15167236328125 -146.6420135498047 -138.3762664794922 -138.37625122070312
474.87200927734375 -119.52790069580078 -136.44058227539062 -145.66236877441406 -125.95634460449219 -125.95633697509766
475.1960144042969 -118.86402130126953 -136.90997314453125 -147.55718994140625 -129.9114990234375 -129.91148376464844
477.58203125 -118.91619873046875 -136.51210021972656 -135.14212036132812 -146.29425048828125 -146.2942657470703
476.96002197265625 -118.75484466552734 -136.40818786621094 -141.23934936523438 -141.595703125 -141.595703125
477.73602294921875 -119.00421142578125 -136.37574768066406 -123.55919647216797 -132.6

475.8060302734375 -117.3343276977539 -135.7582550048828 -117.61235809326172 -134.14395141601562 -134.14393615722656
476.572021484375 -117.4439468383789 -135.1409912109375 -139.37796020507812 -121.90357208251953 -121.90357971191406
473.8440246582031 -115.84695434570312 -134.5457305908203 -128.27639770507812 -141.3642578125 -141.36427307128906
475.5160217285156 -113.50260162353516 -133.82278442382812 -131.582763671875 -137.40386962890625 -137.40386962890625
475.91802978515625 -113.81367492675781 -133.22561645507812 -130.14431762695312 -143.21649169921875 -143.21649169921875
474.32403564453125 -113.27615356445312 -132.1218719482422 -137.44821166992188 -137.07672119140625 -137.0767364501953
475.4040222167969 -111.12335205078125 -131.02964782714844 -143.1667022705078 -117.9443359375 -117.9443359375
475.99603271484375 -110.72599792480469 -130.18612670898438 -116.58932495117188 -113.81703186035156 -113.81702423095703
473.8260192871094 -108.07534790039062 -129.41461181640625 -112.2367858886718

476.0260314941406 -107.1775894165039 -127.5571517944336 -122.70855712890625 -136.74765014648438 -136.74765014648438
473.3280334472656 -107.52568054199219 -127.57351684570312 -119.95671081542969 -135.73617553710938 -135.7362060546875
472.0400085449219 -107.79163360595703 -128.61788940429688 -139.34007263183594 -124.95552062988281 -124.95553588867188
476.3680114746094 -109.2914810180664 -129.5164031982422 -144.11598205566406 -118.0303955078125 -118.03038024902344
475.1300354003906 -110.56273651123047 -129.9866943359375 -139.6248779296875 -117.41993713378906 -117.4199447631836
477.9420166015625 -111.87290954589844 -131.28060913085938 -138.83499145507812 -124.1456069946289 -124.14562225341797
475.7120361328125 -112.48957061767578 -132.54580688476562 -116.09994506835938 -142.54771423339844 -142.54769897460938
476.5160217285156 -114.36864471435547 -133.35357666015625 -138.0036163330078 -146.16384887695312 -146.163818359375
475.8880310058594 -115.32329559326172 -134.01327514648438 -139.682891

480.030029296875 -125.09378051757812 -140.47547912597656 -147.04254150390625 -130.3365936279297 -130.3365936279297
479.86602783203125 -124.23080444335938 -140.4957733154297 -146.29443359375 -133.8443603515625 -133.8443603515625
480.5680236816406 -124.4832992553711 -140.25852966308594 -132.14427185058594 -142.08010864257812 -142.08010864257812
480.8700256347656 -123.87772369384766 -140.2682342529297 -137.91571044921875 -146.300048828125 -146.300048828125
477.20001220703125 -123.17671203613281 -139.9962615966797 -133.4735107421875 -139.21966552734375 -139.21963500976562
476.27801513671875 -123.42869567871094 -139.58163452148438 -145.26992797851562 -125.89013671875 -125.8901138305664
478.82403564453125 -121.82543182373047 -139.19761657714844 -142.06362915039062 -134.86532592773438 -134.86534118652344
477.384033203125 -121.13880157470703 -138.52719116210938 -144.26211547851562 -145.85040283203125 -145.85044860839844
476.9580078125 -120.56843566894531 -138.27410888671875 -144.74765014648438

475.5180358886719 -111.0398941040039 -130.6012725830078 -137.40911865234375 -103.77496337890625 -103.77497100830078
474.468017578125 -110.32850646972656 -129.97198486328125 -131.92922973632812 -138.48196411132812 -138.48196411132812
474.4220275878906 -109.77772521972656 -129.60523986816406 -141.84153747558594 -119.42591857910156 -119.42591094970703
471.2820129394531 -108.74574279785156 -129.03005981445312 -125.07832336425781 -132.55136108398438 -132.55137634277344
472.1920166015625 -109.15609741210938 -129.16534423828125 -121.9928970336914 -115.1138916015625 -115.11389923095703
473.7640075683594 -109.5073013305664 -129.2136993408203 -123.59307861328125 -114.3445053100586 -114.34451293945312
474.39801025390625 -108.98274230957031 -129.41400146484375 -135.31671142578125 -141.7877197265625 -141.7877197265625
474.8580322265625 -110.29557037353516 -130.4092254638672 -121.16719818115234 -127.50227355957031 -127.50228118896484
473.96002197265625 -109.23457336425781 -130.39535522460938 -122.50